# MTurk: Approve / Reject Assignments + Post-Task Completion Processing of HITs

In [1]:
import boto3
import json
import pandas as pd
import ast
from simple_colors import *
from typing import Dict
import re
import pprint
import string
from pathlib import Path


In [2]:
from mturk_processing_utils import *

In [3]:
raw_data_paths = {
    'atomic': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-atomic/',
    'snli': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-snli/',
    'social': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-social/',
}

### Atomic 

In [7]:
# get_completetion_progress(hit_id_2_example_id)

atomic_batches = [
    'mturk_data/creation/atomic_dnli_annotation_examples_1.json',
    'mturk_data/creation/atomic_dnli_annotation_examples_2.json',
    'mturk_data/creation/atomic_dnli_annotation_examples_3.json'
]

dnli_atomic = get_dataset(raw_data_paths['atomic'], 'atomic')

atomic_hit_dicts = [get_hit_id_dict(b)[1] for b in atomic_batches]
atomic_example_dicts = [get_hit_id_dict(b)[2] for b in atomic_batches]

atomic_approved = [
   parse_batch(hit_dict)[0] for hit_dict in atomic_hit_dicts
]

Unique premise-hypothesis pairs: 17689 / 35002
Loaded 35002 nonempty train examples...(skipped 4034 examples)
Unique premise-hypothesis pairs: 1940 / 3840
Loaded 3840 nonempty dev examples...(skipped 470 examples)
Unique premise-hypothesis pairs: 2091 / 4138
Loaded 4138 nonempty test examples...(skipped 516 examples)


In [8]:
atomic_approved_paraphrases = {}

for a in atomic_approved:
    atomic_approved_paraphrases.update(approved_parsed_batch_2_dicts(a, dnli_atomic))

print(len(atomic_approved_paraphrases))

175


#### To Repost

### SNLI

In [4]:
snli_batches = [
    'mturk_data/creation/snli_dnli_annotation_examples_1.json',
    'mturk_data/creation/snli_dnli_annotation_examples_2.json',
    'mturk_data/creation/snli_dnli_annotation_examples_3.json'
]

dnli_snli = get_dataset(raw_data_paths['snli'], 'snli')

snli_hit_dicts = [get_hit_id_dict(b)[1] for b in snli_batches]
snli_example_dicts = [get_hit_id_dict(b)[2] for b in snli_batches]
snli_approved = [
   parse_batch(hit_dict)[0] for hit_dict in snli_hit_dicts
]

Unique premise-hypothesis pairs: 9572 / 88676
Loaded 88676 nonempty train examples...(skipped 5184 examples)
Unique premise-hypothesis pairs: 195 / 1785
Loaded 1785 nonempty dev examples...(skipped 103 examples)
Unique premise-hypothesis pairs: 203 / 1837
Loaded 1837 nonempty test examples...(skipped 135 examples)


In [5]:
snli_approved_paraphrases = {}

for a in snli_approved:
    snli_approved_paraphrases.update(approved_parsed_batch_2_dicts(a, dnli_snli))

print(len(snli_approved_paraphrases))

250


In [ ]:
import json
with open('snli_approved.json', 'w') as f:
    json.dump(paraphrased_dnli_examples, f)

In [ ]:
view_assignment('3EJJQNKU9R5NKFVTHVRTXJCMA43HRZ', dnli_snli, hit_id_2_example_id_snli_batch_3)
response = mturk.create_worker_block(
    WorkerId='APHL3OC024851',
    Reason='Worker is using multiple accounts to fill out verbatim content.'
)